In [ ]:
import numpy as np
from PIL import Image, ImageColor
from google.colab import drive
import skimage.io as io
import numpy as np

In [ ]:
drive.mount('/content/drive')
drive_path = 'drive/My Drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
parts = {10: {'col': 'orange', 'name':'hood'},
         20: {'col':'darkgreen', 'name':'front door'},
         30: {'col':'yellow', 'name':'rear door'},
         40: {'col':'cyan', 'name':'frame'},
         50: {'col':'purple', 'name':'rear quarter panel'},
         60: {'col':'lightgreen', 'name':'trunk lid'},
         70: {'col':'blue', 'name':'fender'},
         80: {'col':'pink', 'name':'bumper'},
         90: {'col':'darkgray', 'name':'rest of car'},
         0 : {'col':'black', 'name':'background'}}

def display_car(data_arr):
    # Can take both full data and already split data
    if type(data_arr) == torch.Tensor: data_arr = np.moveaxis(data_arr.numpy().astype(np.uint8), 0, 2)
    elif data_arr.shape[0] == 3: data_arr = np.moveaxis(data_arr.astype(np.uint8), 0, 2)
    elif data_arr.shape[2] > 3: data_arr = data_arr[:,:,:3]
    img = Image.fromarray(data_arr)
    display(img) # img.show() for jupyter

def display_labels(data_arr):
    # Can take both full data and already split data
    if type(data_arr) == torch.Tensor: data_arr = data_arr.numpy()
    if data_arr.dtype != np.uint8: data_arr = data_arr.astype(np.uint8)*10
    if data_arr.ndim > 2: data_arr = data_arr[:,:,3]
    img = Image.fromarray(data_arr)
    pixels = list(img.getdata())
    pixels = [ImageColor.getrgb(parts.get(pixel)['col']) for pixel in pixels]
    image = Image.new("RGB", (256, 256), (0,0,0))
    image.putdata(pixels)
    display(image)

def numpy_to_tensor(arr):
    return np.moveaxis(arr, 2, 0).astype(np.float32)

def tensor_to_numpy(tens):
    arr = np.moveaxis(tens, 0, 2).astype(np.uint8)
    return arr

In [ ]:
def add_background(car, labels, background):
    mask = (labels == 0)
    updated_car = np.where(mask[..., None], background, car)
    updated_car = updated_car.squeeze()

    return updated_car


In [ ]:
import os
import numpy as np
from torch.utils.data import Dataset
from skimage.transform import resize
from PIL import Image
import random

class CarDataset(Dataset):
    def __init__(self, root, file_list: list=None, backgrounds: list=None):

        self.root = root
        self.filenames = os.listdir(self.root) if file_list is None else file_list
        if backgrounds is not None:
          self.backgrounds = backgrounds
        else:
          self.backgrounds = None

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        arr = np.load(os.path.join(self.root, filename))
        car = arr[:,:,0:3]
        labels = arr[:,:,3]

        if self.backgrounds is not None:
          background = random.choices(self.backgrounds)
          car = add_background(car, labels, background)

        car = car.astype(np.float32)
        car = np.moveaxis(car, 2, 0)

        return car, labels/10

In [ ]:
import os
import numpy as np
from PIL import Image

def load_images_from_folder(folder_path, resize_shape=(256, 256), limit=100):
    image_list = []
    count = 0

    for filename in os.listdir(folder_path):
        # Check if the file is an image file (you can add more extensions if needed)
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            file_path = os.path.join(folder_path, filename)

            # Open the image using PIL
            img = Image.open(file_path)

            # Resize the image
            img = img.resize(resize_shape)

            # Convert the image to a numpy array and append to the list
            image_array = np.array(img)
            image_list.append(image_array)

            count += 1
            if count >= limit:
                break

    return image_list

# Example usage:
folder_path = 'drive/My Drive/carseg_data/images/landscapes'
backgrounds = load_images_from_folder(folder_path, limit=250)


In [ ]:
black_car = []
orange_car = []
photos = []
for file in os.listdir(f'{drive_path}carseg_data/arrays'):
    if 'orange' in file: orange_car.append(file)
    elif 'black' in file: black_car.append(file)
    elif 'photo' in file: photos.append(file)

print(len(black_car), len(orange_car), len(photos))

919 2187 189


In [ ]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

root = f'{drive_path}carseg_data/arrays'

photo_train, photo_test = train_test_split(photos, test_size=0.4, random_state=42)
black_train, black_test = train_test_split(black_car, test_size=0.4, random_state=42)
orange_train, orange_test = train_test_split(orange_car, test_size=0.4, random_state=42)

photo_val, photo_test = train_test_split(photo_test, test_size=0.5, random_state=42)
black_val, black_test = train_test_split(black_test, test_size=0.5, random_state=42)
orange_val, orange_test = train_test_split(orange_test, test_size=0.5, random_state=42)


joint_train_ds = CarDataset(root, photo_train*3+black_train+orange_train)
joint_train_background_ds = CarDataset(root, black_train+orange_train, backgrounds)
joint_val_ds = CarDataset(root, photo_val+black_val+orange_val)

joint_test_ds = CarDataset(root, photo_test+black_test+orange_test)
photo_test_ds = CarDataset(root, photo_test)
black_test_ds = CarDataset(root, black_test)
orange_test_ds = CarDataset(root, orange_test)


train_loader = DataLoader(joint_train_ds+joint_train_background_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(joint_val_ds, batch_size=16)
test_loader = DataLoader(joint_test_ds, batch_size=16)

photo_test_loader =  DataLoader(photo_test_ds, batch_size=16)
black_test_loader =  DataLoader(black_test_ds, batch_size=16)
orange_test_loader =  DataLoader(orange_test_ds, batch_size=16)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
#!pip install torchmetrics
from torchmetrics.classification import Dice

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        return self.block(x)

class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(UNet, self).__init__()
        self.encoder0 = nn.Sequential(ConvBlock(in_channels, 64))
        self.encoder1 = nn.Sequential(nn.MaxPool2d(2,2), ConvBlock(64, 128))
        self.encoder2 = nn.Sequential(nn.MaxPool2d(2,2), ConvBlock(128, 256))
        self.bottleneck = nn.Sequential(nn.MaxPool2d(2,2), ConvBlock(256,512), nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2))
        self.decoder0 = nn.Sequential(ConvBlock(512,256), nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2))
        self.decoder1 = nn.Sequential(ConvBlock(256,128), nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2))
        self.decoder2 = nn.Sequential(ConvBlock(128,64), nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1))

    def forward(self, x):
        x0 = self.encoder0(x)
        x1 = self.encoder1(x0)
        x2 = self.encoder2(x1)
        x3 = self.bottleneck(x2)
        x3 = self.decoder0(torch.cat([x2,x3],dim=1))
        x3 = self.decoder1(torch.cat([x1,x3],dim=1))
        x3 = self.decoder2(torch.cat([x0,x3],dim=1))

        return x3

In [ ]:
device = "cuda"
model = UNet(3, 10).to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [ ]:
num_epochs = 75
dice = Dice(average='micro')
loaders = [train_loader, ]

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0
    dice_scores_train = []


    for batch in train_loader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        inputs = inputs.float()
        labels = labels.long().to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        total_train_loss += loss.item()

        # Calculate dice
        _, pred = torch.max(outputs, 1)
        pred_cpu = pred.to('cpu')
        labels_cpu = labels.to('cpu')

        dice_scores_train.append(dice(pred_cpu, labels_cpu))


        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    total_val_loss = 0.0
    dice_scores_val = []

    for batch in val_loader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        inputs = inputs.float()
        labels = labels.long().to(device)

        with torch.no_grad():
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

            # Calculate dice
            _, pred = torch.max(outputs, 1)
            pred_cpu = pred.to('cpu')
            labels_cpu = labels.to('cpu')

            dice_scores_val.append(dice(pred_cpu, labels_cpu))


    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {(total_train_loss / len(train_loader)):.4f}, Train dice: {np.mean(dice_scores_train):.4f}, Val Loss: {(total_val_loss / len(val_loader)):.4f}, Val dice: {np.mean(dice_scores_val):.4f}")


Epoch 1/75, Train Loss: 0.6951, Train dice: 0.7975, Val Loss: 0.4974, Val dice: 0.8361
Epoch 2/75, Train Loss: 0.5097, Train dice: 0.8218, Val Loss: 0.4335, Val dice: 0.8449
Epoch 3/75, Train Loss: 0.4723, Train dice: 0.8341, Val Loss: 0.3841, Val dice: 0.8641
Epoch 4/75, Train Loss: 0.4438, Train dice: 0.8444, Val Loss: 0.4414, Val dice: 0.8468
Epoch 5/75, Train Loss: 0.4141, Train dice: 0.8552, Val Loss: 0.3650, Val dice: 0.8614
Epoch 6/75, Train Loss: 0.3784, Train dice: 0.8693, Val Loss: 0.3392, Val dice: 0.8742
Epoch 7/75, Train Loss: 0.3607, Train dice: 0.8755, Val Loss: 0.2924, Val dice: 0.8884
Epoch 8/75, Train Loss: 0.3351, Train dice: 0.8835, Val Loss: 0.2737, Val dice: 0.8972
Epoch 9/75, Train Loss: 0.3116, Train dice: 0.8913, Val Loss: 0.2699, Val dice: 0.8975
Epoch 10/75, Train Loss: 0.2926, Train dice: 0.8992, Val Loss: 0.2377, Val dice: 0.9109
Epoch 11/75, Train Loss: 0.2668, Train dice: 0.9074, Val Loss: 0.2266, Val dice: 0.9124
Epoch 12/75, Train Loss: 0.2496, Train di

In [ ]:
def test_model(model, loader):
  """Test a model on a test dataset"""
  dice = Dice(average='micro')
  model.eval()
  dice_scores = []

  for batch in loader:
      inputs, labels = batch
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)

      inputs = inputs.float()
      labels = labels.long().to(device)

      with torch.no_grad():
          outputs = model(inputs)

          # Calculate accuracy on the test set
          _, pred = torch.max(outputs, 1)

          # Move tensors to CPU before performing numpy operations
          pred_cpu = pred.to('cpu')
          labels_cpu = labels.to('cpu')

          dice_scores.append(dice(pred_cpu, labels_cpu))

  return np.mean(dice_scores)



In [ ]:
test_model(model, photo_test_loader)

0.8418195

In [ ]:
test_model(model, black_test_loader)

0.9842939

In [ ]:
test_model(model, orange_test_loader)

0.9854847

In [ ]:
test_model(model, test_loader)

0.97684497

In [ ]:
torch.save(model.state_dict(), f'{drive_path}carseg_data/model2.pth')

## Original

Photos: 0.8259544\
Black: 0.9816798\
Orange: 0.98628855\
Test: 0.9728253

# Added photos

Photos: 0.83027714\
Black: 0.9769864\
Orange: 0.98165745\
Test: 0.9720461

## Added photos and background

Photos: 0.8418195\
Black: 0.9842939\
Orange: 0.9854847\
Test: 0.97684497

